In [1]:
from os import getenv
from dotenv import load_dotenv
import graphviz 
# from relation_algorithms.relation_extraction_functions import LLM_Relation_Extractor
from src.extractor import relationExtractor
import os
import pandas as pd
from src.utils import normalize_text

load_dotenv()

dsa_2214_link = getenv('dsa_2214')
dsa_6114_link = getenv('dsa_6114')
hadoop_link = getenv('hadoop')
token = getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = token

/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


In [2]:
data = pd.read_csv('data/sorting.csv')
data.columns = ['concept', 'outcome']
data

,concept,outcome
0,Data Structures->Basics->Algorithm Analysis,Apply time complexity analysis guideline to an...
1,Data Structures->Basics->Algorithm Analysis->O...,Demonstrate an understanding of big O notation...
2,Algorithms->Non-recursive Algorthims,Demonstrate an understanding of non-recursive ...
3,Algorithms->Non-recursive Algorithms->Search,Apply the Comparable interface for object comp...
4,Algorithms->Non-recursive Algorithms->Search->...,Demonstrate an understanding of linear search;...
5,Algorithms->Non-recursive Algorithms->Search->...,Demonstrate an understanding of binary search;...
6,Algorithms->Non-recursive Algorithms->Sort,Demonstrate an understanding of sorting;
7,Algorithms->Non-recursive Algorithms->Sort->In...,Demonstrate an understanding of insertion sort...
8,Algorithms->Non-recursive Algorithms->Sort->Se...,Demonstrate an understanding of selection sort...
9,Algorithms->Non-recursive Algorithms->Sort->Bu...,Demonstrate an understanding of bubble sort;An...


In [3]:
concept_data = data['concept'].tolist()
actual_concepts = []
for string in concept_data:
    words = string.split('->')
    for word in words:
        if word not in actual_concepts:
            actual_concepts.append(word)


outcome_data = data['outcome'].tolist()
actual_outcomes = []
for string in outcome_data:
    words = string.split(';')
    for s in words:
        if s not in actual_outcomes:
            actual_outcomes.append(s)

In [4]:
dsa_2214_chapters = [
    'Data Structures and Algorithms',
    'Mathematical Preliminaries',
    'Algorithm Analysis',
    'Lists, Stacks, and Queues',
    'Binary Trees',
    'Non-Binary Trees',
    'Internal Sorting',
    'File Processing and External Sorting',
    'Searching',
    'Indexing',
    'Graphs',
    'Lists and Arrays Revisited',
    'Advanced Tree Structures',
    'Analysis Techniques',
    'Lower Bounds',
    'Patterns of Algorithms',
    'Limits to Computation',
]

In [5]:
actual_concepts = [' '.join(actual_concepts)] * 4
actual_outcomes = [' '.join(actual_outcomes)] * 4

from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.embeddings import LangchainEmbeddingsWrapper
from src.transformerEmbeddings import TransformerEmbeddings

eval_llm = LangchainLLMWrapper(langchain_llm = ChatOpenAI(model = 'gpt-4o-mini'))
eval_embeddings = LangchainEmbeddingsWrapper(embeddings = TransformerEmbeddings(model = 'all-MiniLM-L12-v2'))

from ragas.metrics import ResponseRelevancy, LLMContextRecall, SemanticSimilarity, Faithfulness, NoiseSensitivity, LLMContextPrecisionWithReference

In [6]:
metrics = [ResponseRelevancy(), LLMContextPrecisionWithReference(), LLMContextRecall(), SemanticSimilarity(), Faithfulness(), NoiseSensitivity()]

<h4>Testing new retrieval class design... (PLEASE WORK)</h4>

In [7]:
from src.extractor import relationExtractor
extractor = relationExtractor(dsa_2214_link, 
                            token, 
                            dsa_2214_chapters[6:10], 
                            getenv('connection_string'),
                            3000,
                            100,
                            'DocumentEmbeddings',
                            '2214_embeddings',
                            st_model = 'all-MiniLM-L6-v2',
                            reset = True)

In [8]:
import torch
torch.cuda.empty_cache()

In [9]:
retrieved, concepts = extractor.identify_concepts(5)

/home/zanehutchens/uncc/research/rag/src/extractor.py:303: DeprecationWarning: invalid escape sequence '\s'
  relation = re.sub(re.compile('^[a-zA-Z\s\.,!?]'), '', relation)


RuntimeError: CUDA out of memory. Tried to allocate 240.00 MiB (GPU 0; 3.71 GiB total capacity; 2.26 GiB already allocated; 148.75 MiB free; 2.67 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Concepts evaluation without text normalization

In [ ]:
samples = extractor.evaluate('concepts', 5, concepts, actual_concepts, metrics = metrics, data = retrieved)

Concepts evaluation with text normalization on retrieved contexts and concepts

In [ ]:
normalized_concepts = [[normalize_text(' '.join(t))] for t in concepts]
normalized_truths = [normalize_text(t) for t in actual_concepts]

normalized_retrieved = {}
for k in retrieved.keys():
    normalized_retrieved[k] = normalize_text(retrieved[k])

# for i in range(5):
normalized_samples = extractor.evaluate('concepts', 5, normalized_concepts, normalized_truths, data = normalized_retrieved, metrics = metrics)

Outcome evaluation using concepts without text normalization

In [ ]:
outcomes, outcome_contexts = extractor.identify_outcomes(5)

In [ ]:
samples = extractor.evaluate('outcomes', 5, outcomes, actual_outcomes, metrics = metrics, data = retrieved)

Outcome evaluation with text normalization on outcomes and retrieved contexts

In [ ]:
normalized_outcomes = [[normalize_text(' '.join(t))] for t in outcomes]
normalized_truths = [normalize_text(t) for t in actual_outcomes]

normalized_retrieved = {}
for k in retrieved.keys():
    normalized_retrieved[k] = normalize_text(outcome_contexts[k])

samples = extractor.evaluate('outcomes', 5, normalized_outcomes, normalized_truths, data = normalized_retrieved, metrics = metrics)

In [ ]:
chp4_concepts = []
with open('data/chp4.txt', 'r') as f:
    for l in f.readlines():
        concepts = l.strip().split('->')
        for c in concepts:
            if c not in chp4_concepts: chp4_concepts.append(c)

print(chp4_concepts)

In [ ]:
extractor.build_terminology([[word] for word in chp4_concepts])